In [1]:
import tushare as ts
import pandas as pd

token = '82f17f80a6f62681bcbf105689d892a9559a4e65af4045adb472eaf0'
ts.set_token(token)

In [2]:
start_date = "2000-01-01"
end_date = "2015-01-01"

df1 = ts.pro_bar('600000.SH', adj='qfq',
                 start_date=start_date, end_date=end_date)
df1["trade_date"] = pd.to_datetime(df1["trade_date"])
df1.sort_values(by="trade_date", inplace=True)
df1.reset_index(drop=True, inplace=True)
df1["return_rate"] = df1["pct_chg"]/100

df1.drop(["pre_close", "pct_chg", "change"], axis=1, inplace=True)
df1.head()

,ts_code,trade_date,open,high,low,close,vol,amount,return_rate
0,600000.SH,2000-01-04,3.19,3.30,3.16,3.27,44961.0,113946.783,0.0348
1,600000.SH,2000-01-05,3.27,3.32,3.22,3.23,52528.0,134465.427,-0.0122
2,600000.SH,2000-01-06,3.22,3.36,3.20,3.32,62297.0,160059.798,0.0279
3,600000.SH,2000-01-07,3.36,3.52,3.34,3.44,213553.0,575751.082,0.0361
4,600000.SH,2000-01-10,3.45,3.56,3.42,3.48,165397.0,450453.518,0.0116


In [3]:
# 技术指标
from Index import TechnicalIndicator

myindicator = TechnicalIndicator()
df1 = myindicator.getIndex(df1)
df1.reset_index(drop=True, inplace=True)

df1.head()

,ts_code,trade_date,open,high,low,close,vol,amount,return_rate,macd,roc,middleboll,upboll,downboll,cci,rsi,wr,k,d,j
0,600000.SH,2000-02-29,3.22,3.27,3.17,3.22,109559.0,275768.296,0.0031,0.001481,0.625000,3.1845,3.364029,3.004971,4.541768,51.410825,55.172414,33.330350,29.637525,40.716001
1,600000.SH,2000-03-01,3.22,3.47,3.22,3.30,417051.0,1096015.251,0.0248,0.013680,-5.172414,3.1950,3.375817,3.014183,103.543914,56.948933,44.736842,40.641720,33.305808,55.313546
2,600000.SH,2000-03-02,3.30,3.32,3.21,3.25,129999.0,330633.290,-0.0152,0.014678,-0.914634,3.2010,3.380696,3.021304,30.613333,52.842420,57.894737,41.129587,35.913838,51.561087
3,600000.SH,2000-03-03,3.27,3.41,3.27,3.32,161098.0,420418.261,0.0215,0.022851,3.105590,3.2120,3.392596,3.031404,159.041859,57.520655,39.473684,46.864321,39.564095,61.464772
4,600000.SH,2000-03-06,3.39,3.44,3.27,3.27,206162.0,542130.542,-0.0151,0.020702,2.830189,3.2210,3.393767,3.048233,167.748746,53.393247,55.555556,46.057681,41.728662,54.715720


In [4]:
import numpy as np

pro = ts.pro_api()
#  市值、换手率、杠杆因子
df_daily_basic = pro.daily_basic(ts_code='600000.SH', start_date=start_date, end_date=end_date,
                                 fields="trade_date,total_mv,pe_ttm,turnover_rate_f")
df_daily_basic["trade_date"] = pd.to_datetime(df_daily_basic["trade_date"])
df_daily_basic.sort_values("trade_date", inplace=True)

# 市值因子取对数
df_daily_basic["lncap"] = np.log(df_daily_basic["total_mv"])
del df_daily_basic["total_mv"]

# 过去20天换手率（流通股本）
df_daily_basic["turnover_rate_rolling"] = df_daily_basic["turnover_rate_f"].rolling(20).mean()
del df_daily_basic["turnover_rate_f"]

df_daily_basic.dropna(inplace=True)
df_daily = pd.merge(df1, df_daily_basic, on="trade_date", how="inner")

df_daily.head()

,ts_code,trade_date,open,high,low,close,vol,amount,return_rate,macd,...,downboll,cci,rsi,wr,k,d,j,pe_ttm,lncap,turnover_rate_rolling
0,600000.SH,2000-02-29,3.22,3.27,3.17,3.22,109559.0,275768.296,0.0031,0.001481,...,3.004971,4.541768,51.410825,55.172414,33.330350,29.637525,40.716001,70.5807,15.618205,3.623260
1,600000.SH,2000-03-01,3.22,3.47,3.22,3.30,417051.0,1096015.251,0.0248,0.013680,...,3.014183,103.543914,56.948933,44.736842,40.641720,33.305808,55.313546,72.4034,15.643702,4.202110
2,600000.SH,2000-03-02,3.30,3.32,3.21,3.25,129999.0,330633.290,-0.0152,0.014678,...,3.021304,30.613333,52.842420,57.894737,41.129587,35.913838,51.561087,71.2817,15.628088,4.325320
3,600000.SH,2000-03-03,3.27,3.41,3.27,3.32,161098.0,420418.261,0.0215,0.022851,...,3.031404,159.041859,57.520655,39.473684,46.864321,39.564095,61.464772,72.9081,15.650649,4.449870
4,600000.SH,2000-03-06,3.39,3.44,3.27,3.27,206162.0,542130.542,-0.0151,0.020702,...,3.048233,167.748746,53.393247,55.555556,46.057681,41.728662,54.715720,71.7864,15.635145,4.627465


In [5]:
# 财务数据
df_fina = pro.fina_indicator(ts_code='600000.SH', start_date=start_date, end_date=end_date,
                             fields='end_date,roe,q_gr_yoy,debt_to_assets')
df_fina["end_date"] = pd.to_datetime(df_fina["end_date"])
for k in range(len(df_fina)):
    mydate = df_fina.loc[k, "end_date"]
    while True:
        if mydate in df_daily["trade_date"].tolist():
            break
        else:
            mydate = mydate + pd.Timedelta(days=1)
    df_fina.loc[k, "end_date"] = mydate
df_fina.head()

,end_date,roe,debt_to_assets,q_gr_yoy
0,2014-12-31,20.2461,93.7252,21.3160
1,2014-09-30,15.9734,94.0733,18.3790
2,2014-06-30,10.7200,94.3688,28.2940
3,2014-03-31,5.0870,94.1281,25.4009
4,2013-12-31,21.4323,94.3690,24.4824


In [6]:
# 财务数据合并,如果开始早，保留最早的一个
df_fina.drop_duplicates(subset=["end_date"], keep="first", inplace=True)
df_all = pd.merge(df_daily, df_fina, left_on="trade_date",
                  right_on="end_date", how="outer")
df_all.drop(["end_date"], inplace=True, axis=1)
df_all.fillna(method="bfill", inplace=True)
df_all.dropna(inplace=True)
df_all.head()

,ts_code,trade_date,open,high,low,close,vol,amount,return_rate,macd,...,wr,k,d,j,pe_ttm,lncap,turnover_rate_rolling,roe,debt_to_assets,q_gr_yoy
0,600000.SH,2000-02-29,3.22,3.27,3.17,3.22,109559.0,275768.296,0.0031,0.001481,...,55.172414,33.330350,29.637525,40.716001,70.5807,15.618205,3.623260,6.5944,92.6751,53.0924
1,600000.SH,2000-03-01,3.22,3.47,3.22,3.30,417051.0,1096015.251,0.0248,0.013680,...,44.736842,40.641720,33.305808,55.313546,72.4034,15.643702,4.202110,6.5944,92.6751,53.0924
2,600000.SH,2000-03-02,3.30,3.32,3.21,3.25,129999.0,330633.290,-0.0152,0.014678,...,57.894737,41.129587,35.913838,51.561087,71.2817,15.628088,4.325320,6.5944,92.6751,53.0924
3,600000.SH,2000-03-03,3.27,3.41,3.27,3.32,161098.0,420418.261,0.0215,0.022851,...,39.473684,46.864321,39.564095,61.464772,72.9081,15.650649,4.449870,6.5944,92.6751,53.0924
4,600000.SH,2000-03-06,3.39,3.44,3.27,3.27,206162.0,542130.542,-0.0151,0.020702,...,55.555556,46.057681,41.728662,54.715720,71.7864,15.635145,4.627465,6.5944,92.6751,53.0924


In [7]:
# 滚动回归，计算beta,alpha,e_var
def get_beta_e(s1, s2):
    chg_my = s1
    sh_chg = s2
    model = np.polyfit(x=chg_my, y=sh_chg, deg=1)
    beta = model[0]  # beta
    alpha = model[1]  # alpha
    get_array = np.array([model[0] * my_ + model[1] for my_ in sh_chg])
    e_array = chg_my - get_array
    var_e = np.sum(e_array ** 2) / (len(e_array) - 2)  # e2的无偏估计
    return alpha, beta, var_e


df2 = ts.pro_bar('600000.SH', adj='qfq',
                 start_date=start_date, end_date=end_date)
df2["trade_date"] = pd.to_datetime(df2["trade_date"])
df2["return_rate"] = df2["pct_chg"]/100
df2 = df2[["trade_date", "return_rate"]]

sh_res = ts.pro_bar(ts_code='000001.SH', adj='qfq', asset="I",
                    start_date=start_date, end_date=end_date)
sh_array = sh_res["pct_chg"] / 100
sh_df = pd.DataFrame({"date": sh_res["trade_date"], "sh_chg": sh_array})
sh_df["date"] = pd.to_datetime(sh_df["date"])

df_tmp = pd.merge(df2, sh_df, left_on="trade_date",
                  right_on="date", how="inner")

df_tmp.sort_values(by="trade_date", inplace=True)
df_tmp.reset_index(inplace=True, drop=True)

# rolling.apply 似乎无法输出多列？
regress_day = 20
for j in range(len(df_all) - regress_day - 1):
    s1 = np.array(df_tmp.loc[j + 1:j + regress_day + 1, "return_rate"])
    s2 = np.array(df_tmp.loc[j + 1:j + regress_day + 1, "sh_chg"])
    alpha, beta, var_e = get_beta_e(s1, s2)
    df_tmp.loc[j + regress_day, "alpha"] = alpha
    df_tmp.loc[j + regress_day, "beta"] = beta
    df_tmp.loc[j + regress_day, "var_e"] = var_e

df_tmp = df_tmp[["trade_date", "sh_chg", "alpha", "beta", "var_e"]]
df_tmp[20:25]

,trade_date,sh_chg,alpha,beta,var_e
20,2000-02-15,-0.001955,0.009426,0.765525,0.000572
21,2000-02-16,0.013431,0.007853,0.786779,0.000491
22,2000-02-17,-0.030984,0.008063,0.769073,0.000532
23,2000-02-18,0.016726,0.006169,0.678952,0.000548
24,2000-02-21,0.005422,0.005764,0.683083,0.000582


In [8]:
# 合并滚动回归的数据
df_all = pd.merge(df_all, df_tmp, on="trade_date", how="inner")
df_all.dropna(inplace=True)
df_all.head()

,ts_code,trade_date,open,high,low,close,vol,amount,return_rate,macd,...,pe_ttm,lncap,turnover_rate_rolling,roe,debt_to_assets,q_gr_yoy,sh_chg,alpha,beta,var_e
0,600000.SH,2000-02-29,3.22,3.27,3.17,3.22,109559.0,275768.296,0.0031,0.001481,...,70.5807,15.618205,3.623260,6.5944,92.6751,53.0924,0.005703,0.006491,0.605899,0.000568
1,600000.SH,2000-03-01,3.22,3.47,3.22,3.30,417051.0,1096015.251,0.0248,0.013680,...,72.4034,15.643702,4.202110,6.5944,92.6751,53.0924,-0.009818,0.006919,0.594308,0.000609
2,600000.SH,2000-03-02,3.30,3.32,3.21,3.25,129999.0,330633.290,-0.0152,0.014678,...,71.2817,15.628088,4.325320,6.5944,92.6751,53.0924,0.008994,0.006530,0.587057,0.000603
3,600000.SH,2000-03-03,3.27,3.41,3.27,3.32,161098.0,420418.261,0.0215,0.022851,...,72.9081,15.650649,4.449870,6.5944,92.6751,53.0924,0.014598,0.005095,0.616446,0.000565
4,600000.SH,2000-03-06,3.39,3.44,3.27,3.27,206162.0,542130.542,-0.0151,0.020702,...,71.7864,15.635145,4.627465,6.5944,92.6751,53.0924,-0.032759,0.004717,0.617602,0.000551


In [9]:
df_all.to_csv("../datasets/feature_train.csv", index=False)